In [21]:
import requests
import pandas as pd
import datetime

from joblib import load, dump
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

from evidently.metrics import ColumnQuantileMetric
from evidently.report import Report
from evidently.ui.workspace import Workspace
from evidently.ui.dashboards import DashboardPanelCounter, DashboardPanelPlot, CounterAgg, PanelValue, PlotType, ReportFilter
from evidently.renderers.html_widgets import WidgetSize

In [8]:
files = [('green_tripdata_2024-03.parquet', './data')]

print('Downloading files...')
for file, path in files:
    url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{file}'
    response = requests.get(url, stream=True)
    save_path = f'{path}/{file}'
    with open(save_path, 'wb') as f:
        for data in tqdm(response.iter_content(),
                         desc=f'{file}',
                         postfix=f'Save to {save_path}',
                         total=int(response.headers.get('content-length'))):
            f.write(data)
march_data = pd.read_parquet('./data/green_tripdata_2024-03.parquet')

green_tripdata_2024-03.parquet: 100%|██████████| 1372372/1372372 [00:08<00:00, 160768.18it/s, Save to ./data/green_tripdata_2024-03.parquet]


In [9]:
print(f"Q1: {march_data.shape}")

Q1: (57457, 20)


In [10]:
print("Q2: ColumnQuantileMetric")

Q2: ColumnQuantileMetric


In [27]:
report = Report(metrics=[
    ColumnQuantileMetric(column_name="fare_amount", quantile=0.5)
])

report.run(reference_data=None, current_data=march_data)
result = report.as_dict()
q3 = result['metrics'][0]['result']['current']['value']
print(f"Q3: {q3}")

Q3: 13.5


In [28]:
ws = Workspace("workspace")
project = ws.create_project("Homework week 5")
project.description = "My project for homework week 5"
project.save()
ws.add_report(project.id, report)

In [29]:
project.dashboard.add_panel(
    DashboardPanelPlot(
        filter=ReportFilter(metadata_values={}, tag_values=[]),
        title="Inference Count",
        values=[
            PanelValue(
                metric_id="ColumnQuantileMetric",
                field_path="current.value",
                legend="count"
            ),
        ],
        plot_type=PlotType.BAR,
        size=WidgetSize.HALF,
    ),
)
project.save()

Project(id=UUID('019a81f5-1f99-74ed-963d-06af7bf23f1e'), name='Homework week 5', description='My project for homework week 5', dashboard=DashboardConfig(name='Homework week 5', panels=[DashboardPanelPlot(type='evidently:dashboard_panel:DashboardPanelPlot', id=UUID('019a81f5-2676-7a8b-9f81-f8bc2a3a7333'), title='Inference Count', filter=ReportFilter(metadata_values={}, tag_values=[], include_test_suites=False), size=<WidgetSize.HALF: 1>, values=[PanelValue(field_path='current.value', metric_id='ColumnQuantileMetric', metric_fingerprint=None, metric_args={}, legend='count')], plot_type=<PlotType.BAR: 'bar'>)], tabs=[], tab_id_to_panel_ids={}), team_id=None, org_id=None, date_from=None, date_to=None, created_at=datetime.datetime(2025, 11, 14, 12, 41, 56, 633750), version='1')

In [30]:
print(f'Q4: ./dashboards')

Q4: ./dashboards
